In [1]:
# prompt: pip install catboost

%pip install catboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
#from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb


In [27]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [28]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [29]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [11]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [33]:
X.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [12]:
scalar=StandardScaler()
X = scalar.fit_transform(X)
X_testdata = scalar.transform(X_testdata)




In [ ]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split


# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# # Initialize and train the LightGBM model on full feature set to get feature importances
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     metric='binary_logloss',
#     boosting_type='gbdt',
#     num_leaves=20,
#     max_depth=4,
#     learning_rate=0.03,
#     n_estimators=376,
#     subsample=0.7,
#     colsample_bytree=0.5,
#     min_child_weight=5,
#     scale_pos_weight=1.1,
#     reg_lambda=0.1,
#     verbose=-1
# )

# # Fit the model on full data to get feature importances
# clf.fit(X_train, y_train)

# # Get feature importances and create a DataFrame
# feature_importances = clf.feature_importances_
# feature_importances_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# })

# # Sort the DataFrame by importance
# feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# # Define different numbers of top features to test
# feature_counts = [10, 15, 20, 25, 30,35,40,45,50,55,60]
# auc_scores = {}

# # Iterate over different numbers of top features
# for n in feature_counts:
#     # Select the top n features
#     top_features = feature_importances_df['Feature'].head(n).tolist()

#     # Subset the training and testing data to the top n features
#     X_train_top = X_train[top_features]
#     X_test_top = X_test[top_features]

#     # Re-train the model with the top n features
#     clf.fit(X_train_top, y_train)

#     # Predict probabilities on the test set
#     y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

#     # Calculate AUC score
#     auc = roc_auc_score(y_test, y_pred_proba)
#     auc_scores[n] = auc  # Store the AUC score for this number of features

#     print(f"Top {n} features - AUC: {auc:.4f}")

# # Print all AUC scores for comparison
# print("\nAUC scores for different feature counts:")
# for n, auc in auc_scores.items():
#     print(f"Top {n} features: AUC = {auc:.4f}")


Top 10 features - AUC: 0.9437
Top 15 features - AUC: 0.9500
Top 20 features - AUC: 0.9586
Top 25 features - AUC: 0.9619
Top 30 features - AUC: 0.9608
Top 35 features - AUC: 0.9625
Top 40 features - AUC: 0.9632
Top 45 features - AUC: 0.9609
Top 50 features - AUC: 0.9622
Top 55 features - AUC: 0.9616
Top 60 features - AUC: 0.9640

AUC scores for different feature counts:
Top 10 features: AUC = 0.9437
Top 15 features: AUC = 0.9500
Top 20 features: AUC = 0.9586
Top 25 features: AUC = 0.9619
Top 30 features: AUC = 0.9608
Top 35 features: AUC = 0.9625
Top 40 features: AUC = 0.9632
Top 45 features: AUC = 0.9609
Top 50 features: AUC = 0.9622
Top 55 features: AUC = 0.9616
Top 60 features: AUC = 0.9640


In [13]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV


# # Define the parameter grid for LightGBM
# param_grid = {
#     'learning_rate': [0.01, 0.03, 0.1],
#     'max_depth': [4, 6, 8, -1],  # -1 indicates no limit for depth in LightGBM
#     'n_estimators': [100, 200, 376, 500],
#     'num_leaves': [20, 31, 40, 50],  # Controls complexity in LightGBM
#     'reg_lambda': [0.01, 0.1, 1],
#     'min_child_weight': [1, 3, 5],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'subsample': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [1, 1.1, 1.2]
# }

# # Initialize the classifier
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     random_state=42
# )

# random_search = RandomizedSearchCV(
#     estimator=clf,
#     param_distributions=param_grid,
#     cv=3,
#     n_iter=50,
#     scoring='roc_auc',
#     verbose=1,
#     n_jobs=-1
# )

# # Fit the model
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", random_search.best_params_)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# !pip install mlxtend
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# # ... (your existing code to prepare X, y, X_testdata) ...


# # Convert X and X_testdata back to DataFrames if they are NumPy arrays
# if isinstance(X, np.ndarray):
#     X = pd.DataFrame(X, columns=train_set.drop(columns=['Y', 'RecordId']).columns)  # Use original column names
# if isinstance(X_testdata, np.ndarray):
#     X_testdata = pd.DataFrame(X_testdata, columns=test_set.drop(columns=['RecordId']).columns)  # Use original column names

# # Calculate correlations with the target variable
# correlations = X.corrwith(y)

# # Select features with correlation value greater than 0.01
# selected_features = correlations[correlations.abs() > 0.01].index

# # Print the selected features
# print("Selected Features (Correlation > 0.01):")
# for feature in selected_features:
#     print(f"{feature} (Correlation: {correlations[feature]:.3f})")

# # Filter X and X_testdata based on selected features
# X = X[selected_features]
# X_testdata = X_testdata[selected_features]

# # Define base classifiers
# base_classifiers = [
#     ('logistic_regression', LogisticRegression()),
#     ('decision_tree', DecisionTreeClassifier()),
#     ('random_forest', RandomForestClassifier()),
#     ('knn', KNeighborsClassifier()),
#     #('svm', SVC()),  # Avoid SVC due to potential long training times
# ]

# # Define meta-classifier
# meta_classifier = LogisticRegression()

# # Create StackingClassifier
# stacking_model = StackingClassifier(
#     estimators=base_classifiers,
#     final_estimator=meta_classifier,
#     cv=5  # 5-fold cross-validation
# )

# # Train the stacking model
# stacking_model.fit(X, y)

# # Make predictions on the test data
# y_pred = stacking_model.predict(X_testdata)

Selected Features (Correlation > 0.01):
X2 (Correlation: 0.021)
X4 (Correlation: -0.011)
X6 (Correlation: -0.019)
X10 (Correlation: 0.016)
X12 (Correlation: 0.017)
X14 (Correlation: -0.019)
X15 (Correlation: -0.018)
X16 (Correlation: -0.010)
X17 (Correlation: -0.062)
X18 (Correlation: -0.073)
X19 (Correlation: 0.017)
X20 (Correlation: -0.053)
X21 (Correlation: 0.021)
X22 (Correlation: 0.039)
X24 (Correlation: 0.021)
X25 (Correlation: -0.040)
X26 (Correlation: 0.050)
X28 (Correlation: -0.042)
X30 (Correlation: -0.041)
X31 (Correlation: 0.034)
X34 (Correlation: -0.045)
X36 (Correlation: -0.044)
X37 (Correlation: 0.026)
X40 (Correlation: -0.050)
X42 (Correlation: -0.050)
X44 (Correlation: -0.049)
X46 (Correlation: -0.052)
X48 (Correlation: -0.028)
X50 (Correlation: -0.028)
X51 (Correlation: 0.025)
X52 (Correlation: 0.017)
X53 (Correlation: -0.013)
X54 (Correlation: -0.030)
X55 (Correlation: -0.012)
X56 (Correlation: -0.031)
X57 (Correlation: 0.028)
X58 (Correlation: 0.025)
X59 (Correlatio

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Initialize the individual models with their best parameters
lgb_model = lgb.LGBMClassifier(
    objective='binary',
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=7,
    num_leaves=175,
    min_child_samples=25,
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    learning_rate=0.03,
    max_depth=6,
    n_estimators=500,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42
)

catboost_model = CatBoostClassifier(
    loss_function='Logloss',
    eval_metric='AUC',
    iterations=3000,
    learning_rate=0.03,
    depth=8,
    l2_leaf_reg=3,
    border_count=64,
    random_strength=1.5,
    bagging_temperature=1,
    task_type='CPU'
)

# Train each model
lgb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train)

# Get feature importances
lgb_importances = lgb_model.feature_importances_
xgb_importances = xgb_model.feature_importances_
catboost_importances = catboost_model.feature_importances_

# Average feature importances
avg_importances = np.mean([lgb_importances, xgb_importances, catboost_importances], axis=0)

# Select top features
top_features = np.argsort(avg_importances)[-60:]  # Select top 10 features

# Reduce the dataset to the top features
X_train_selected = X_train[:, top_features]
X_test_selected = X_test[:, top_features]
X_testdata_selected=X_testdata[:, top_features]
# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('lgb', lgb_model),
    ('xgb', xgb_model),
    ('catboost', catboost_model)
], voting='soft', weights=[4, 1, 4])

# Train the voting classifier
voting_clf.fit(X_train_selected, y_train)

# Make predictions
y_pred = voting_clf.predict(X_test_selected)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))




[LightGBM] [Info] Number of positive: 536, number of negative: 196361
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17734
[LightGBM] [Info] Number of data points in the train set: 196897, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.002722 -> initscore=-5.903576
[LightGBM] [Info] Start training from score -5.903576
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [30]:
y_pred = voting_clf.predict(X_test_selected)
y_pred_proba = voting_clf.predict_proba(X_test_selected)[:, 1] # Get probabilities for the positive class


# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_proba)) # Calculate and print AUC

Accuracy: 0.9977653631284916
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     49112
           1       1.00      0.03      0.05       113

    accuracy                           1.00     49225
   macro avg       1.00      0.51      0.53     49225
weighted avg       1.00      1.00      1.00     49225

AUC: 0.9083982863081964


In [31]:
# Predict probabilities for the test set
y_pred_proba_testdata = voting_clf.predict_proba(X_testdata_selected)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities_vote.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[5.94600630e-05 2.11165909e-03 1.81523980e-06 ... 3.62416934e-02
 2.34788946e-03 1.31949851e-03]
        RecordId  Y_probability
0         300001       0.000059
1         300002       0.002112
2         300003       0.000002
3         300004       0.000138
4         300005       0.000004
...          ...            ...
105477    405478       0.000002
105478    405479       0.246454
105479    405480       0.000057
105480    405481       0.000006
105481    405482       0.000008

[105482 rows x 2 columns]
